# Example: Single Agent() 

Install SUMO, SMARTS, required packages, and build the scenarios.

In [5]:
# Setup SUMO=1.10.0
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev xorg
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo

# Install SMARTS
%cd /content/
!git clone https://github.com/huawei-noah/SMARTS
%cd /content/SMARTS 
!git checkout develop
%pip install -e .[camera-obs]
import sys
sys.path.insert(0, '/content/SMARTS/')

# Build scenarios
!scl scenario build-all --clean /content/SMARTS/scenarios/loop

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
  Using cached eclipse_sumo-1.10.0-py2.py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (49.8 MB)
Note: you may need to restart the kernel to use updated packages.
env: SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo
[Errno 2] No such file or directory: '/content/'
/home/adai/workspaces/smarts-2/examples
Cloning into 'SMARTS'...
remote: Enumerating objects: 21487, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100

Define the agent and the environment loop.

In [6]:
import gym

from smarts.core.agent import Agent
from smarts.core.sensors import Observation

class ChaseViaPointsAgent(Agent):
    def act(self, obs: Observation):
        if (
            len(obs.via_data.near_via_points) < 1
            or obs.ego_vehicle_state.road_id != obs.via_data.near_via_points[0].road_id
        ):
            return (obs.waypoint_paths[0][0].speed_limit, 0)

        nearest = obs.via_data.near_via_points[0]
        if nearest.lane_index == obs.ego_vehicle_state.lane_index:
            return (nearest.required_speed, 0)

        return (
            nearest.required_speed,
            1 if nearest.lane_index > obs.ego_vehicle_state.lane_index else -1,
        )

pybullet build time: Oct  8 2020 00:10:46


In [16]:
from smarts.core.agent import AgentSpec
from smarts.core.agent_interface import AgentInterface, AgentType
from smarts.core.utils.episodes import episodes
from smarts.env.wrappers.single_agent import SingleAgent

AGENT_ID = "SingleAgent"

def main(scenarios, sim_name, headless, num_episodes, seed, max_episode_steps=None):
    agent_spec = AgentSpec(
        interface=AgentInterface.from_type(
            AgentType.LanerWithSpeed, max_episode_steps=max_episode_steps
        ),
        agent_builder=ChaseViaPointsAgent,
    )

    env = gym.make(
        "smarts.env:hiway-v0",
        scenarios=scenarios,
        agent_specs={AGENT_ID: agent_spec},
        sim_name=sim_name,
        headless=headless,
        visdom=False,
        fixed_timestep_sec=0.1,
        sumo_headless=True,
        seed=seed,
    )

    # Wrap a single-agent env with SingleAgent wrapper to make `step` and `reset`
    # output compliant with gym spaces.
    env = SingleAgent(env)

    for episode in episodes(n=num_episodes):
        agent = agent_spec.build_agent()
        observation = env.reset()
        episode.record_scenario(env.scenario_log)

        done = False
        while not done:
            agent_action = agent.act(observation)
            observation, reward, done, info = env.step(agent_action)
            episode.record_step(observation, reward, done, info)

    env.close()

Run the example

In [17]:
!scl envision start -s /content/SMARTS/scenarios

Traceback (most recent call last):
  File "/home/adai/workspaces/SMARTS/examples/intersection/.venv/bin/scl", line 5, in <module>
    from cli.cli import scl
  File "/home/adai/workspaces/SMARTS/examples/intersection/.venv/lib/python3.7/site-packages/cli/cli.py", line 27, in <module>
    from cli.studio import scenario_cli
  File "/home/adai/workspaces/SMARTS/examples/intersection/.venv/lib/python3.7/site-packages/cli/studio.py", line 29, in <module>
    from smarts.core.sumo_road_network import SumoRoadNetwork
  File "/home/adai/workspaces/SMARTS/examples/intersection/.venv/lib/python3.7/site-packages/smarts/core/sumo_road_network.py", line 39, in <module>
    from .sumo_lanepoints import LinkedLanePoint, SumoLanePoints
  File "/home/adai/workspaces/SMARTS/examples/intersection/.venv/lib/python3.7/site-packages/smarts/core/sumo_lanepoints.py", line 43, in <module>
    from smarts.core.utils.sumo import sumolib  # isort:skip
  File "/home/adai/workspaces/SMARTS/examples/intersection/.v

%%html
<iframe src="https://localhost:8081" width="100" height="100"></iframe>

In [18]:
main(
    scenarios="/content/SMARTS/scenarios/loop",
    sim_name="Single agent",
    headless=True,
    num_episodes=10,
    seed=42,
)

ModuleNotFoundError: No module named 'sumolib'